In [1]:
import numpy as np
import json
from PIL import Image
import torch
import torch.nn as nn

from torch.optim import lr_scheduler
from torchvision import transforms
import matplotlib.pyplot as plt

import torch.optim as optim

import time
import os
import copy
import random
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import cv2
from tqdm.auto import tqdm

from efficientnet_pytorch import EfficientNet

In [2]:
CFG = {
    'data_path' : 'train',
    'batch' : 16,
    'epoch' : 100,
    'model' : 'efficientnet-b5',
    'num_classes' : 88,
    'img_size' : 456,
    'random_seed' : 555
    
}

# 데이터 호출

In [3]:
df = pd.read_csv('train_df_aug.csv')

label = df['label']
le  = LabelEncoder().fit(label)

label_ec = le.transform(label)

encdf = pd.DataFrame(label_ec,columns = ['encode'])
data = pd.concat([df['file_name'],encdf],axis=1)

In [4]:
from glob import glob

def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    
    # get image path
    img_path_list.extend(glob(os.path.join(data_dir, '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('\\')[-1].split('.')[0]))
    label_list.extend(data['encode'])
                
    return img_path_list, label_list

def get_test_data(data_dir):
    img_path_list = []
    
    img_path_list.extend(glob(os.path.join(data_dir, '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('\\')[-1].split('.')[0]))

    
    return img_path_list

In [5]:
all_img_path, all_label = get_train_data('aug')
test_img_path = get_test_data('test')

In [6]:
all_img_path[0]

'aug\\20.png'

In [7]:
import torchvision.datasets as datasets 
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index): 
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image)

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self): 
        return len(self.img_path_list)

In [8]:
train_len = int(len(all_img_path)*0.75)
Vali_len = int(len(all_img_path)*0.25)

train_img_path = all_img_path[:train_len]
train_label = all_label[:train_len]

vali_img_path = all_img_path[train_len:]
vali_label = all_label[train_len:]

print('train set 길이 : ', train_len)
print('vaildation set 길이 : ', Vali_len)

train set 길이 :  4602
vaildation set 길이 :  1534


In [9]:

train_transform = transforms.Compose([
                    transforms.ToPILImage(), 
                    transforms.Resize([CFG['img_size'], CFG['img_size']]),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) 
                    
                    ])

test_transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize([CFG['img_size'], CFG['img_size']]),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) 
                    ])

In [10]:
train_dataset = CustomDataset(train_img_path, train_label, train_mode=True, transforms=train_transform) 
vali_dataset = CustomDataset(vali_img_path, vali_label, train_mode=True, transforms=test_transform)
test_dataset = CustomDataset(vali_img_path, vali_label, train_mode=True, transforms=test_transform)

In [11]:
dataloaders, batch_num = {}, {}
dataloaders['train'] = torch.utils.data.DataLoader(train_dataset,
                                              batch_size=CFG['batch'], shuffle=True,
                                              num_workers=0)
dataloaders['valid'] = torch.utils.data.DataLoader(vali_dataset,
                                              batch_size=CFG['batch'], shuffle=False,
                                              num_workers=0)
dataloaders['test']  = torch.utils.data.DataLoader(vali_dataset,
                                              batch_size=CFG['batch'], shuffle=False,
                                              num_workers=0)
batch_num['train'], batch_num['valid'], batch_num['test'] = len(dataloaders['train']), len(dataloaders['valid']), len(dataloaders['test'])
print('batch_size : %d,  tvt : %d / %d / %d' % (CFG['batch'], batch_num['train'], batch_num['valid'], batch_num['test']))

batch_size : 16,  tvt : 288 / 96 / 96


In [12]:
image_size = EfficientNet.get_image_size(CFG['model'])
print(image_size)
model = EfficientNet.from_pretrained(CFG['model'], num_classes=CFG['num_classes'])

456
Loaded pretrained weights for efficientnet-b5


In [13]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_loss, train_acc, valid_loss, valid_acc = [], [], [], []
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss, running_corrects, num_cnt = 0.0, 0, 0
            
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                num_cnt += len(labels)
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = float(running_loss / num_cnt)
            epoch_acc  = float((running_corrects.double() / num_cnt).cpu()*100)
            
            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                valid_loss.append(epoch_loss)
                valid_acc.append(epoch_acc)
            print('{} Loss: {:.2f} Acc: {:.1f}'.format(phase, epoch_loss, epoch_acc))
           
            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_idx = epoch
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
#                 best_model_wts = copy.deepcopy(model.module.state_dict())
                print('==> best model saved - %d / %.1f'%(best_idx, best_acc))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: %d - %.1f' %(best_idx, best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'president_model.pt')
    print('model saved')
    return model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc

In [14]:
class FocalLoss(nn.Module):

    def __init__(self, gamma=5.0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = nn.CrossEntropyLoss(reduction="none")

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # set gpu

model = model.to(device)

criterion = FocalLoss()



optimizer_ft = optim.SGD(model.parameters(), 
                         lr = 5e-4,
                         momentum=0.9,
                         weight_decay=1e-4)

lmbda = lambda epoch: 0.98739
exp_lr_scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer_ft, lr_lambda=lmbda)



In [ ]:
model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=CFG['epoch'])

Epoch 0/99
----------


In [ ]:
def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.to(device)

            pred_logit = model(img)
            pred_logit = pred_logit.argmax(dim=1, keepdim=True).squeeze(1)

            model_pred.extend(pred_logit.tolist())
    return model_pred

test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['batch'], shuffle=False, num_workers=0)

# Inference
preds = predict(model, test_loader, device)
preds[0:5]

In [ ]:
answer = le.inverse_transform(preds)
submission = pd.read_csv('test.csv')
submission['label'] = answer

label_split = df['label'].str.split("-")
submission["class"] = label_split.str.get(0)
submission["state"] = label_split.str.get(1)

submit = submission[['file_name','class','state','label']]

submit.to_csv('submit.csv', index=False)

In [ ]:
print(len(preds ))